In [5]:
import arcpy
import os

# --------------------------------------------------
# Setup
# --------------------------------------------------
aprx = arcpy.mp.ArcGISProject("CURRENT")
m = aprx.listMaps()[0]   # use the active map

# --------------------------------------------------
# Locate elevation layer (already added to map)
# --------------------------------------------------
lyr = None
for l in m.listLayers():
    if "boundary" in l.name.lower():   
        lyr = l
        break

if lyr is None:
    raise Exception("Boundary shapefile layer not found in the map.")

# --------------------------------------------------
# Apply graduated color symbology on elevation
# --------------------------------------------------
sym = lyr.symbology
sym.updateRenderer("GraduatedColorsRenderer")
sym.renderer.classificationField = "elevation"
sym.renderer.breakCount = 5
sym.renderer.classificationMethod = "NaturalBreaks"
lyr.symbology = sym

print("Graduated color symbology applied on elevation.")

# --------------------------------------------------
# Helper function: rectangle in page units (inches)
# --------------------------------------------------
def make_rect_ll(llx, lly, w, h):
    xy = [
        [llx, lly],
        [llx, lly + h],
        [llx + w, lly + h],
        [llx + w, lly],
        [llx, lly]
    ]
    arr = arcpy.Array([arcpy.Point(*coords) for coords in xy])
    return arcpy.Polygon(arr)

# --------------------------------------------------
# Create layout (Letter portrait)
# --------------------------------------------------
lyt = aprx.createLayout(8.5, 11, "INCH", "Elevation Layout")
print("Layout created.")

# --------------------------------------------------
# Create map frame
# --------------------------------------------------
mf = lyt.createMapFrame(
    make_rect_ll(0.75, 1.0, 7.0, 9.0),
    m,
    "Map Frame"
)
print("Map frame added.")

# --------------------------------------------------
# Add north arrow
# --------------------------------------------------
na_style = aprx.listStyleItems(
    "ArcGIS 2D",
    "North_Arrow",
    "Compass North 1"
)[0]

north_arrow = lyt.createMapSurroundElement(
    arcpy.Point(6.5, 9.1),
    "North_Arrow",
    mf,
    na_style,
    "North Arrow"
)
north_arrow.elementWidth = 0.5
print("North arrow added.")

# --------------------------------------------------
# Add scale bar
# --------------------------------------------------
sb_style = aprx.listStyleItems(
    "ArcGIS 2D",
    "Scale_bar",
    "Double Alternating Scale Bar 1 Metric"
)[0]

scale_bar = lyt.createMapSurroundElement(
    make_rect_ll(4.75, 0.5, 3.5, 0.5),
    "Scale_bar",
    mf,
    sb_style,
    "Scale Bar"
)
print("Scale bar added.")

# --------------------------------------------------
# Add legend
# --------------------------------------------------
leg_style = aprx.listStyleItems(
    "ArcGIS 2D",
    "LEGEND",
    "Legend 3"
)[0]

legend = lyt.createMapSurroundElement(
    arcpy.Point(1.0, 9.5),
    "LEGEND",
    mf,
    leg_style,
    "Legend"
)
legend.elementWidth = 2.2
legend.elementHeight = 2.2
legend.title = "Elevation (m)"
legend.columnCount = 1
legend.fittingStrategy = "AdjustFontSize"

print("Legend added.")

# --------------------------------------------------
# Export layout to PDF
# --------------------------------------------------
out_pdf = r"C:\Users\thany\Documents\programming\geog4057_TanyaSutthanonkul\TermProject\elevation_layout.pdf"
lyt.exportToPDF(out_pdf, resolution=300)

aprx.save()
print(f"Layout exported to: {out_pdf}")
print("Project saved.")

Graduated color symbology applied on elevation.
Layout created.
Map frame added.
North arrow added.
Scale bar added.
Legend added.
Layout exported to: C:\Users\thany\Documents\programming\geog4057_TanyaSutthanonkul\TermProject\elevation_layout.pdf
Project saved.
